In [96]:

text = open('2023_19.txt', 'r').read()
text1 = '''px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}'''

def parse_data(data):
    # Split the data on the comma
    parts = data.split(',')

    # Initialize an empty list to hold the final result
    result = []

    # For each part, split it on the colon and append the result to the final list
    for part in parts:
        # Check if the part contains '<' or '>'
        if '<' in part:
            operator = '<'
        elif '>' in part:
            operator = '>'
        else:
            # If the part doesn't contain an operator, just append it to the result and continue with the next part
            result.append([part])
            continue

        subparts = part.split(':')
        # Split the first element of subparts on the operator to separate the operator from the value
        operator_value = subparts[0].split(operator)
        # Replace the first element of subparts with the result of the split
        subparts[0] = operator_value[1]
        # Prepend the operator to the subparts
        subparts.insert(0, operator_value[0] + operator)
        # Append the subparts to the result
        result.append(subparts)

    return result
# convert the text rows which contains {R,U,L,D}, and integer and (RGBcolor) to a list of lists
# [[R,6,(#70c710)], [D,5,(#0dc571)], ...]
x=[]
m=[]
a=[]
s=[]
data_dict = {}

lines = text.split('\n')
# Parse each line
for line in lines:
    if (line == ""):
        continue
    if (line[0] == '{'):
        parts = line.replace('{', '').replace('}', '').split(',')
        # For each part, split it into a variable name and a value, and add the value to the appropriate list
        for part in parts:
            var, val = part.split('=')
            exec(f'{var}.append({val})')
    else:
        # Split the line into a key and a value at the first '{'
        key, value = line.split('{', 1)
        # Remove the trailing '}' from the value
        value = value.rstrip('}')
        # Add the key-value pair to the dictionary
        data_dict[key] = parse_data(value)

def apply_rule(rule,part,xmas):
    if rule == 'R':
        return 0
    elif rule == 'A':
        return 1
    elif len(data_dict[rule][part]) == 1:
        return apply_rule(data_dict[rule][part][0],0,xmas)
    else:
        #find index of char in data_dict[rule][part][0] in 'xmas'
        ind = 'xmas'.index(data_dict[rule][part][0][0])
        if data_dict[rule][part][0][1] == '<':
            if xmas[ind] < int(data_dict[rule][part][1]):
                return apply_rule(data_dict[rule][part][2],0,xmas)
            else:
                return apply_rule(rule,part+1,xmas)
        elif data_dict[rule][part][0][1] == '>':
            if xmas[ind] > int(data_dict[rule][part][1]):
                return apply_rule(data_dict[rule][part][2],0,xmas)
            else:
                return apply_rule(rule,part+1,xmas)
        else:
            print('error')
            return -1
    return -1

res = [0]*len(x)
# for each part in the list, check that the first condition is true, if so, move to the next rule, or A=accept,R=reject
for ind in range(len(x)):
    xmas = [x[ind], m[ind], a[ind], s[ind]]
    res[ind] = apply_rule(rule='in',part=0,xmas=xmas)

tot = 0
for ind in range(len(x)):
    if res[ind] == 1:
        tot += x[ind] + m[ind] + a[ind] + s[ind]

print('Part 1:',tot)


Part 1: 487623


In [98]:
xmasmin = [0,0,0,0] 
xmasmax = [4001,4001,4001,4001]
leaf = []
def pushrule(name,part,xmasmin,xmasmax):
    #print(name,part,xmasmin,xmasmax)
    if name == 'R':
        return
    elif name == 'A':
        leaf.append([xmasmin,xmasmax])
        return
    cond = data_dict[name][part]
    if len(cond) == 1:
        pushrule(cond[0],0,xmasmin[:],xmasmax[:])
        return
    else:   
        #find index of char in data_dict[rule][part][0] in 'xmas'
        ind = 'xmas'.index(cond[0][0])
        if cond[0][1] == '<':
            val = xmasmax[ind]
            xmasmax[ind] = min(xmasmax[ind],int(cond[1]))
            pushrule(cond[2],0,xmasmin[:],xmasmax[:])
            xmasmax[ind] = val
            xmasmin[ind] = max(xmasmin[ind],int(cond[1])-1)
            pushrule(name,part+1,xmasmin[:],xmasmax[:])
            return
        elif cond[0][1] == '>':
            val = xmasmin[ind]
            xmasmin[ind] = max(xmasmin[ind],int(cond[1]))
            pushrule(cond[2],0,xmasmin[:],xmasmax[:])
            xmasmin[ind] = val
            xmasmax[ind] = min(xmasmax[ind],int(cond[1])+1)
            pushrule(name,part+1,xmasmin[:],xmasmax[:])
            return
        else:
            print('error')
            return -1
pushrule('in', 0, xmasmin, xmasmax)

total=0
for i in range(len(leaf)):
    perms = 1
    for j in range(4):
        perms *= (leaf[i][1][j] - leaf[i][0][j]-1)
    total += perms
print('Part 2:',total)


Part 2: 113550238315130
